<a href="https://colab.research.google.com/github/NICE-MSI/NPL-Academy/blob/main/NPL_NiCE-MSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Using Mass Spectrometry Imaging to Map Molecules**

In this notebook, you will be able to investigate different mean spectra from some tissues of interest. You will overplot spectra from the different tissues, study the intensity ratios of different compounds of interest, as well as .....

First, we need to import the python packages that are needed to read and plot the data. (numpy, matplotlib, pandas):

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

We clone the Mass Spectrometry Imaging data that we are going to study in this Notebook.

In [ ]:
!git clone https://github.com/NICE-MSI/NPL-Academy.git

We use "pandas" (a python package) to read the file with the mean spectra of the different tissues. 
We can print the file on the next cell to see its structure.

In [ ]:
df = pd.read_csv("NPL-Academy/spectra.csv")  # read data files
print(df)  #print data file

As you can see, there are 6 columns in the file. The first column corresponds to the m/z values (X-axis of the spectrum). Columns 2 to 6 correspond to the intensities of the spectra for the different tissues (Y-axis).

Note: You can save your figures by "uncomment" the last line (removing #), but you need to comment #plt.show() for the figure to be saved. Your plots will be saved in the "ouputs" folder.

In [ ]:
plt.plot(df["m/z"],df["A_APCKRAS"], color='blue', label='tissue A-APCKRAS')
plt.plot(df["m/z"],df["D_APCKRAS"], color='red', label='tissue D-APCKRAS')
plt.legend()
plt.show()
#plt.savefig("NPL-Academy/outputs/filename.png")


Can you overplot all the spectra in "spectra.csv"? Remeber to include the labels on your plot.
You can customize your plot in many different ways (colors, linestyles, linewidth,...). If you want to look at all the options you can check here:
https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html 

You can also zoom-in into specific areas of the spectrum for a better visualisation. For example:

In [ ]:
plt.plot(df["m/z"],df["A_APCKRAS"], color='blue', label='tissue A-APCKRAS')
plt.xlim((300,320))
plt.legend()
plt.show()
#plt.savefig("NPL-Academy/outputs/filename.png")

NOISE DETERMINATION

One of the first problems we have when analysing MSI data, is to differentiate signal from noise. MSI normally contains big amount of data, so it is important to save only the compounds of interest, and remove the noise. In this example, we can determine noise level in a basic way, obtaining the standard deviation of the mean spectrum. 

In the next cell we obtain the standard deviation of two of the tissues (A and D) by using the "std" function in the numpy package (np.std)

In [ ]:
print('standard deviation for tissue A =', np.std(df["A_APCKRAS"]))
print('standard deviation for tissue D =',np.std(df["D_APCKRAS"]))

We can use this standard deviation as a threshold to determine signal and noise.
In the cell below we are plotting the noise in red and the signal in blue.

In [ ]:
treshold = np.std(df["A_APCKRAS"])
plt.plot(df["m/z"],df["A_APCKRAS"].where(df["A_APCKRAS"]<treshold), color='red', label='noise A-APCKRAS')
plt.plot(df["m/z"],df["A_APCKRAS"].where(df["A_APCKRAS"]>=treshold), color='blue', label='signal A-APCKRAS')
plt.legend()
plt.show()
#plt.savefig("NPL-Academy/outputs/filename.png")

Do you think this noise level is correct? 
Let's zoom-in at one specific area to have a better visualisation.

In [ ]:
treshold = np.std(df["A_APCKRAS"])

plt.plot(df["m/z"],df["A_APCKRAS"].where(df["A_APCKRAS"]<treshold), color='red', label='noise A-APCKRAS')
plt.plot(df["m/z"],df["A_APCKRAS"].where(df["A_APCKRAS"]>=treshold), color='blue', label='signal A-APCKRAS')
plt.xlim((310,320))
plt.ylim((-1E5,6E5))
plt.legend()
plt.show()
#plt.savefig("NPL-Academy/outputs/filename.png")

Can you lower the noise level for this spectrum? Use the cell above to determin the noise level that you think could be best in this case. 

Can you investigate what happens in other areas of the spectrum when you use different tresholds? Can you use the same threshold for all the spectrum to remove noise properly?

Remeber to save some plots for your presentation on Friday.

INTENSITY RATIOS

To study what compounds are more relevant in each type of tissues (i.e. APC or APCKRAS), 
we can study the intensity ratios between peaks of the mean spectra of these tissues.
We provide you here with a list of 70 common peaks among the tissues. 
This file has 7 columns, the m/z value and the intensities for each of the 6 tissues we are working with.

In [ ]:
peaks = pd.read_csv("NPL-Academy/top70_peaks.csv")  # read data files
print(peaks)  #print data file

To study the ratio between the intensity of the compounds for the APC vs APCKRAS tissues, we create the mean of each of the APCKRA tissues and the APC tissues separately, and then we obtain its ratio.
In the next cell we creat a new column in the table called "mean APCKRAS". Can you add another column called "mean APC"? 
Note: Python numerates the first column of the table as 0. You can see the names of the columns using 
print(peaks.columns)

In [ ]:
peaks['mean APCKRAS'] = peaks.iloc[:, [1,5,6]].mean(axis=1)

print(peaks)

Once you have the two new columns, We can compare their ratios, and create a new column called 'ratio'.  

In [ ]:
peaks['ratio'] = peaks['mean APCKRAS']/peaks['mean APC']

We can select the top 10 ions with ratio > in APCKRAS.

In [ ]:

print(peaks.nlargest(10,'ratio'))

Can you identify the top 10 ions with ration > in APC?

You can study the single ion images in the supporting material. 

T-TEST

Another way to study the presence of the different compunds in two different samples is by using the t-test.
For this, we need to load another python package.

In [ ]:
from scipy.stats import ttest_ind

In the next cell we will calculate the p and t-values for each of the 70 compounds in our list. The t score is a ratio between the difference between two groups and the difference within the groups.

Larger t scores = more difference between groups.
Smaller t score = more similarity between groups.

A p-value from a t test is the probability that the results from your sample data occurred by chance. P-values are from 0% to 100% and are usually written as a decimal (for example, a p value of 5% is 0.05). Low p-values indicate your data did not occur by chance. For example, a p-value of 0.01 means there is only a 1% probability that the results from an experiment happened by chance. 

In [ ]:

t=np.zeros(len(peaks['ratio']))
p=np.zeros(len(peaks['ratio']))

for jj in range(0,len(peaks['ratio'])):
    apckras_intensities = np.array(peaks.iloc[jj, [1,5,6]])
    apc_intensities = np.array(peaks.iloc[jj, [2,3,4]])
    t[jj],p[jj] = ttest_ind(apckras_intensities,apc_intensities,equal_var=False)

peaks['t'] = t.tolist()
peaks['p'] = p.tolist()

print(peaks)


Can you identify the top 10 p-values? (Hint: you can use the function "nsmallest")

What are the p-values of these ions?

Will using a ratio provide different results to using t-test?

In [ ]:
print()

STATISTICS

When doing statistical analysis, it is important to understandand the importance of the uncertainties in the measurements. The results from the intensity ratios and the t-test are different because we are not considering the standard deviation (uncertainty) of the mean when obtaining the ratios. 
We can obtain the uncertainty for then mean of each of the tissues.

In [ ]:
peaks['std APCKRAS'] = peaks.iloc[:, [1,5,6]].std(axis=1)
peaks['std APC'] = peaks.iloc[:, [2,3,4]].std(axis=1)


If we correct from this error in our measurements, we can obtain more similar results than when using the t-test. 

In [ ]:
peaks['ratio mean/std'] = 1/((peaks["std APCKRAS"]/peaks['mean APCKRAS'])**2 +(peaks["std APC"]/peaks['mean APC'])**2)**0.5
print(peaks.nlargest(10,'ratio mean/std'))

Are these results more similar to the t-test results? 

The t-test is a more statistically correct method for this type of analysis.

Remeber that every measurement has an uncertainty associated at it. The bigger the amount of measurement, the smallest the uncertainty will be. There are as well systematic errors associated to the instrumentation. These errors are independent on the number of measurements, they will only be the same and might compromise our results. It is very important to distinguished and identify both types of errors.  

In [ ]:
peaks = pd.read_csv("NPL-Academy/tumour_normal.csv")  # read data files
print(peaks)  #print data file
